### setup

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

import notebook_setup
from typing import Union, Iterable, List
from copy import deepcopy
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from tqdm.autonotebook import tqdm, trange
import optuna

from rl import learn_rl, transform_rl_policy, evaluate_rl, PPO, load_agent
from multirotor.simulation import Multirotor
from multirotor.helpers import DataLog
from multirotor.visualize import plot_datalog
from multirotor.controller import Controller
from multirotor.trajectories import Trajectory, GuidedTrajectory
from multirotor.controller.scurves import SCurveController
from systems.multirotor import MultirotorTrajEnv, VP
from multirotor.controller import (
    AltController, AltRateController,
    PosController, AttController,
    VelController, RateController,
    Controller
)
from scripts.opt_pidcontroller import (
    get_controller, make_disturbance_fn,
    apply_params as apply_params_pid, get_study as get_study_pid
)
from scripts.opt_multirotorenv import apply_params, get_study

## Helper functions

In [ ]:
# PID or PPO
# Alloc or Traj
def run_sim(
    env: MultirotorTrajEnv, traj: Trajectory,
    ctrl: Union[Controller, PPO, None],
    max_steps=2_000, relative=None, verbose=False
) -> DataLog:
    if not isinstance(traj, Trajectory):
        traj = Trajectory(env.vehicle, traj, proximity=env._proximity, resolution=env.bounding_box/2)
    log = DataLog(env.vehicle, ctrl if isinstance(ctrl, Controller) else env.ctrl,
                  other_vars=('reward',))
    if isinstance(ctrl, PPO):
        pidctrl = env.ctrl
        predict_fn = lambda x: ctrl.predict(x, deterministic=True)[0]
    elif ctrl is None or isinstance(ctrl, Controller):
        ctrl = pidctrl = env.ctrl
        predict_fn = lambda x: env.normalize_action(np.zeros(env.action_space.shape, env.vehicle.dtype))
    
    if verbose:
        iterator = tqdm(enumerate(traj), leave=False, total=max_steps // env.steps_u)
    else:
        iterator = enumerate(traj)
    for i, (pos, feed_forward_vel) in iterator:
        # Get prescribed normalized action for system as thrust and torques
        action = predict_fn(env.state)
        # Send speeds to environment
        state, r, done, *_, i = env.step(action)
        log.log(reward=r)
        if done:
            if verbose:
                print(i)
            break

    log.done_logging()
    return log

def run_trajectory(
    env, traj: Union[Trajectory, Iterable], ctrl=None, verbose=False, reset_zero=True, log=None
) -> DataLog:
    if reset_zero:
        env.reset(np.zeros(12)) # resets controller too
    if log is None:
        old_wp = np.zeros(3)
        pos_global = env.vehicle.position
    else:
        old_wp = log.target.position[-1]
        pos_global = log.position[-1]
        env.vehicle.x = log.states[-1]
        env.x = env.vehicle.x
    if not isinstance(traj, Trajectory):
        traj = Trajectory(env.vehicle, traj, proximity=env._proximity, resolution=env.bounding_box/2)
    params = traj.get_params()
    points = traj.generate_trajectory(curr_pos=pos_global)[1:]
    for wp in points:
        wp_rel = wp - old_wp
        pos = env.vehicle.position - wp_rel
        state = np.concatenate((pos, env.vehicle.state[3:]))
        env.reset(state)
        t = Trajectory(env.vehicle, [[0,0,0]], **params)
        l = run_sim(env, t, env.ctrl if ctrl is None else ctrl, verbose=verbose)
        if len(l):
            l.position[:] += old_wp + wp_rel
            l.target.position[:, :3] += old_wp + wp_rel
        else:
            continue
        if log is None:
            log = l
        else:
            log.append(l, relative=True)
        old_wp = wp
    return log

def run_comparison(env_fns, agents, n=5) -> List[List[float]]:
    R, S = [], []
    for ctrl, env_fn in tqdm(zip(agents, env_fns), leave=False, total=len(agents)):
        env = env_fn()
        r = []
        for i in trange(n, leave=False):
            env.reset()
            traj = Trajectory(env.vehicle, [[0,0,0]], proximity=0.1)
            log = run_sim(env, traj, ctrl, verbose=False)
            r.append(log.reward.sum())
        R.append(r)
        S.append(np.std(r))
    return R

def get_log_df(agent):
    return get_tensorboard_scalar_frame(agent.logger.dir)

# Trajectory Env
def get_env(params={}, scurve=False, **kwargs):  
    kw = dict(
        bounding_box=20,
        vp=VP,get_controller_fn=lambda m: get_controller(m, scurve),
        steps_u=params.get('steps_u', 1),
        scaling_factor=params.get('scaling_factor', 1.),
        seed=0)
    kw.update(kwargs)
    return MultirotorTrajEnv(**kw)

def get_wind_quiver(heading: str, ax: plt.Axes, n=5, dim=2):
    angle = float(heading.split('@')[-1]) * np.pi / 180
    dx, dy = -np.cos(angle), -np.sin(angle)
    xlim, ylim = ax.get_xlim(), ax.get_ylim()
    if dim==3:
        zlim = ax.get_zlim()
        dz = 0
        x,y,z = np.meshgrid(np.linspace(*xlim, num=n), np.linspace(*ylim, num=n), np.linspace(*zlim, num=n),
                      indexing='xy')
        return x,y,z,dx,dy,dz
    else:
        x,y = np.meshgrid(np.linspace(*xlim, num=n), np.linspace(*ylim, num=n),
                      indexing='xy')
        return x,y,dx,dy

nasa_wp = np.asarray([
    [0.0, 0.0, 30.0],
    [164.0146725649829, -0.019177722744643688, 30.0],
    [165.6418055187678, 111.5351051245816, 30.0],
    [127.3337449710234, 165.73576059611514, 30.0],
    [-187.28170707810204, 170.33217775914818, 45.0],
    [-192.03130502498243, 106.30660058604553, 45.0],
    [115.89920266153058, 100.8644210617058, 30.0],
    [114.81859536317643, 26.80923518165946, 30.0],
    [-21.459931490011513, 32.60508110653609, 30.0]
])
wp = np.asarray([
    [10, 0, 0],
    [10, 10, 0],
    [0, 10, 0],
    [0, 0, 0]
])
def params_to_labels(params):
    param_dict = {
        'steps_u':'(f_{RL} \delta t)^{-1}',
        'n_epochs':'epochs',
        'pid-p.k_i': 'position k_i',
        'pid-r_pitch_roll.max_acceleration': 'rate max_{acc}',
        'pid-a_pitch_roll.k_p': 'attitude k_p',
        'pid-r_pitch_roll.k_d': 'rate k_d',
        'scaling_factor': 'scaling factor' ,
        'pid-r_pitch_roll.k_i': 'rate k_i',
        'pid-v.k_d': 'velocity k_d',
        'pid-v.k_i': 'velocity k_i',
        'pid-r_pitch_roll.k_p': 'rate k_p',
        'pid-a_pitch_roll.k_i': 'attitude k_i',
        'pid-p.k_d': 'position k_d',
        'pid-v.k_p': 'velocity k_p',
        'pid-a_pitch_roll.k_d': 'attitude k_d',
        'pid-p.k_p': 'position k_p',
        'learning_rate': 'learning rate',
        'n_epochs': 'epochs'
    }
    kind_dict = dict(p='position', v='velocity', a='attitude', r='rate')
    labels = []
    return ['$' + param_dict[p] + '$' for p in params]
            
        

## PID

In [ ]:
study_names_pid = (#'MultirotorPIDWind5@90',
               'MultirotorPIDWind5@0',
               'MultirotorPIDWind5@any',
               'MultirotorPID',)
labels_pid = (#'PID for 90$\degree$',
          'PID for 0$\degree$',
          'PID for random wind',
          'PID for No wind')

def get_study_params_pid(name):
    pidstudy = get_study_pid(name)
    best_pid_params = pidstudy.best_params
    return pidstudy, best_pid_params

nominal_pidstudy, best_pid_params_nominal = get_study_params_pid('MultirotorPID')

pidstudy, best_pid_params = get_study_params_pid('MultirotorPIDWind5@0')

In [ ]:
points = [[0,0,0],[10,0,0],[20,0,0],[20,10,0],[20,20,0]]
traj = Trajectory(None, points, resolution=6)
wp = np.asarray(traj.generate_trajectory())
plt.scatter(wp[:,0], wp[:,1])

In [ ]:
nominal_pidstudy.best_value

In [ ]:
env = get_env({}, scurve=False, seed=0, disturbance_fn=lambda m: np.asarray([0,-0,0]))
# env.ctrl.feedforward_weight = 0.

plot_agent = None
apply_params(env.ctrl, **best_pid_params)
env.reset([0,0,0, 0,0,0, 0,0,0, 0,0,0])
waypoints = np.asarray([[10,10,0]])
traj = Trajectory(env.vehicle, waypoints, resolution=env.bounding_box/2.5, proximity=env._proximity)
log = run_trajectory(env, traj, plot_agent, verbose=False)
plot_datalog(log)

### PID Disturbances

In [ ]:
plot_agent = None

env = get_env({}, scurve=False, seed=0, disturbance_fn=lambda m: np.asarray([1,-1,0]))
apply_params(env.ctrl, **best_pid_params)
env.reset()
traj = Trajectory(env.vehicle, nasa_wp, resolution=env.bounding_box/2.5, proximity=env._proximity)
log1 = run_trajectory(env, traj, plot_agent, verbose=False)

env = get_env({}, scurve=False, seed=0, disturbance_fn=lambda m: np.asarray([0,0,0]))
apply_params(env.ctrl, **best_pid_params)
env.reset()
traj = Trajectory(env.vehicle, nasa_wp, resolution=env.bounding_box/2.5, proximity=env._proximity)
log2 = run_trajectory(env, traj, plot_agent, verbose=False)

plt.plot(log1.x, log1.y, ls='-', label='disturbance')
plt.plot(log2.x, log2.y, ls=':', label='nominal')
plt.legend()

### S-Curves

In [ ]:
from pyscurve.trajectory import plot_trajectory
from pyscurve.scurve import ScurvePlanner
traj = ScurvePlanner().plan_trajectory([0], [100], [0], [2], 10, 10, 10)
plot_trajectory(env.ctrl.traj, 0.01)

### PID Analysis

In [ ]:
# Different RL controllers under the same disturbance
heading = '5@0'
wp = np.asarray([[10,0,0], [10,10,0], [0,10,0], [0,0,0]])
step = 10

for label, study_name in zip(labels_pid, study_names_pid):
    # path = './tensorboard/MultirotorTrajEnv/optstudy/%s/' % study_name
    study = get_study_pid(study_name)
    # Best agent using best test performance
    df = study.trials_dataframe().sort_values('value', ascending=False)
    df = df.reset_index(drop=True)
    # best_trial = df['number'][0]
    # best_agent = load_agent((path + '%03d/run_1/agent') % best_trial)
    
    env = get_env(scurve=False, disturbance_fn=make_disturbance_fn(heading))
    apply_params_pid(env.ctrl, **study.best_params)
    
    log = run_trajectory(env, wp, None)
    x0, x1 = log.x[::step], log.target.position[::step,0]
    y0, y1 = log.y[::step], log.target.position[::step,1]
    plt.plot(log.x,log.y, label=label, ls=':')
    
wx, wy, dx, dy = get_wind_quiver(heading, plt.gca(), n=5)
plt.quiver(wx, wy, dx, dy,
           scale_units='xy', angles='xy', scale=1, units='dots', width=0.75)
plt.scatter(wp[:,0], wp[:,1], marker='x', c='r', s=100)

plt.gca().set_aspect('equal', 'box');
plt.xlabel('x /m')
plt.ylabel('y /m')
plt.title(r'Wind: %sN from %s$\degree$' % (*heading.split('@'),))
plt.legend();

## RL

Fine-tune using `scripts/opt_multirotorenv.py` script. Then load study for analysis.

In [ ]:
study_names = (#'MultirotorTrajEnvPIDWind5@90',
               'MultirotorTrajEnvPIDWind5@0',
               'MultirotorTrajEnv5@any',
               'MultirotorTrajEnvPID',
              )
headings = (#'5@90',
            '5@0',
            '5@any',
            '0@0',
           )
labels = (#'RL for 90$\degree$',
          'RL for 0$\degree$',
          'RL for random wind',
          'RL for No wind',
         )
log_root_path = './tensorboard/MultirotorTrajEnv/optstudy/%s/'

def get_study_agent_params(name):
    study = get_study(name)
    best_trial = study.best_trial.number
    best_agent = load_agent((log_root_path + '%03d/run_1/agent') % (name, best_trial))
    best_params = study.best_params
    return study, best_agent, best_params

In [ ]:
study_name = study_names[3]
study, best_agent, best_params = get_study_agent_params(study_name)

In [ ]:
# Best agent using best last reward during training
from tbparse import SummaryReader
from glob import glob
max_r = -np.inf
max_path = ''
for p in glob(path + '*'):
    S.append(SummaryReader(p, pivot=True))
    r = S[-1].scalars['rollout/ep_rew_mean'].iloc[-1]
    if r > max_r:
        max_path = p

best_agent = load_agent(p + '/run_1/agent')
best_params = study.trials[int(max_path[-3:])].params

In [ ]:
# Fine tune
env = get_env(scurve=False)
apply_params(env, **best_params)
agent = learn_rl(env, steps=100_000,
                 n_steps=study.best_params['n_steps'],
                 batch_size=study.best_params['batch_size'],
                 n_epochs=study.best_params['n_epochs'],
                 learning_rate=study.best_params['learning_rate'],
                 tensorboard_log=env.name + '/tuning/' + study_name,
                 seed=0,
                 reuse_parameters_of=best_agent,
                 log_env_params=('steps_u','scaling_factor'),
                 policy_kwargs=dict(net_arch=[dict(pi=[128,128],
                                                   vf=[128,128])])
                )

In [ ]:
study, agent, params = get_study_agent_params(study_names[1])

In [ ]:
plt.figure()
dfs  = [get_study(name).trials_dataframe() for name in study_names]
df = pd.concat(dfs, ignore_index=True)

df['f_RL'] = 1 / (0.01 * df['params_steps_u'])
df.plot.hexbin('params_scaling_factor', 'f_RL', C='value',
               gridsize=20, figsize=(4,3))
# plt.xlabel()
plt.ylabel('$f_{RL}$')

### RL vs PID

In [ ]:
# trajectory wth setpoints, compared against PID
heading = '5@0'
wp = np.asarray([[10,0,0], [10,10,0], [0,10,0], [0,0,0]])

env_rl = get_env(scurve=False, disturbance_fn=make_disturbance_fn(heading))
apply_params(env_rl, **best_params)

env_pid = get_env(scurve=False, disturbance_fn=make_disturbance_fn(heading))
apply_params_pid(env_pid.ctrl, **best_pid_params)

env_pid_nominal = get_env(scurve=False, disturbance_fn=make_disturbance_fn(heading))
apply_params_pid(env_pid_nominal.ctrl, **best_pid_params_nominal)


logrl = run_trajectory(env_rl, wp, best_agent)
logpid = run_trajectory(env_pid, wp, None)
logpidn = run_trajectory(env_pid_nominal, wp, None)

In [ ]:
plt.figure(figsize=(5,5))
plt.xlim(-5, 15)
plt.ylim(-5, 15)
step = 5

# Cascaded PID only
x0, x1 = logpidn.x[::step], logpidn.target.position[::step,0]
y0, y1 = logpidn.y[::step], logpidn.target.position[::step,1]
l, = plt.plot(logpidn.x,logpidn.y, label='PID-Nominal', ls=':')

# PID-only without RL
x0, x1 = logpid.x[::step], logpid.target.position[::step,0]
y0, y1 = logpid.y[::step], logpid.target.position[::step,1]
plt.plot(logpid.x,logpid.y, label='PID-only', c=l.get_c())

# RL
x0, x1 = logrl.x[::step], logrl.target.position[::step,0]
y0, y1 = logrl.y[::step], logrl.target.position[::step,1]
plt.plot(logrl.x,logrl.y, label='RL+PID')
plt.scatter(logrl.target.position[::step,0], logrl.target.position[::step,1], c='k', zorder=100)
# plt.quiver(x0, y0,
#            x1-x0, y1-y0,
#            headwidth=3, headlength=6,zorder=100, ls=':',
#            scale_units='xy', angles='xy', scale=1, units='dots', width=1.5)

wx, wy, dx, dy = get_wind_quiver(heading, plt.gca(), n=7)
plt.quiver(wx, wy, dx, dy,
           scale_units='xy', angles='xy', scale=1, units='dots', width=0.75)
plt.scatter(wp[:,0], wp[:,1], marker='x', c='r', s=100)

plt.gca().set_aspect('equal', 'box');
plt.xlabel('x /m')
plt.ylabel('y /m')
plt.title(r'Wind: %sN from %s$\degree$' % (*heading.split('@'),))
plt.legend(loc='lower center', ncol=3);

### Robustness

In [ ]:
# Different RL controllers under the same disturbance
plt.figure(figsize=(5,5))
heading = '5@0'
wp = np.asarray([[10,0,0], [10,10,0], [0,10,0], [0,0,0]])
step = 10

for label, study_name in zip(labels[1:-1], study_names[1:-1]):
    print(study_name)
    study, best_agent, best_params = get_study_agent_params(study_name)
    
    env = get_env(scurve=False, disturbance_fn=make_disturbance_fn(heading))
    apply_params(env, **study.best_params)
    
    log = run_trajectory(env, wp, best_agent)
    x0, x1 = log.x[::step], log.target.position[::step,0]
    y0, y1 = log.y[::step], log.target.position[::step,1]
    plt.plot(log.x,log.y, label=label)
    
wx, wy, dx, dy = get_wind_quiver(heading, plt.gca(), n=5)
plt.quiver(wx, wy, dx, dy,
           scale_units='xy', angles='xy', scale=1, units='dots', width=0.75)
plt.scatter(wp[:,0], wp[:,1], marker='x', c='r', s=100)

plt.gca().set_aspect('equal', 'box');
plt.xlabel('x /m')
plt.ylabel('y /m')
plt.title(r'Wind: %sN from %s$\degree$' % (*heading.split('@'),))
plt.legend();

### Relationship between conditions

In [ ]:
# Relationship between RL supervision to disturbances
wp = np.asarray([[10,0,0], [10,10,0], [0,10,0], [0,0,0]])
step = 10
actions = []
for heading, study_name in zip(headings, study_names):
    study, best_agent, best_params = get_study_agent_params(study_name)
    
    best_params = study.best_params
    env_rl = get_env(scurve=False, disturbance_fn=make_disturbance_fn(heading))
    env_rl.reset()
    apply_params(env_rl, **best_params)
    state_samples = np.asarray(
        [env_rl.observation_space.sample() for _ in range(1000)],
        env_rl.vehicle.dtype
    )
    norm_actions = best_agent.predict(state_samples)[0]
    actions.append(env_rl.unnormalize_action(norm_actions))

In [ ]:
# ...continued: Histograms of angles of actions
def rotate_deg(arr: np.ndarray, by: float):
    arr = arr.copy()
    arr += by
    arr[arr<-180] += 360
    arr[arr>180] -= 360
    return arr
plt.figure(figsize=(6,3))
angles = [np.asarray([np.arctan(y/x) * 180 / np.pi for x,y in a[:,:2]]) for a in actions]
corrections = [np.asarray([-180 if x<0 else 0 for x in a[:,0]]) for a in actions]
angles = [a+c for a,c in zip(angles, corrections)]
for a in angles:
    a[a<-180] = 360 + a[a<-180]
    a[a>180] = a[a>180] - 360
plt.hist(angles, histtype='step', density=True, label=labels, lw=2)
plt.xlabel('Angle /$\degree$ of supervisory action on x/y plane')
plt.ylabel('Probability density')
plt.legend(loc='lower center', ncol=3)

In [ ]:
# ...continued: Rotating angles and comparing histograms
plt.figure(figsize=(6,3))
angles_xformed = deepcopy(angles[:2])
angles_xformed[0] = angles_xformed[0] - 90
angles_xformed[1] = angles_xformed[1] + 90
for a in angles_xformed[:2]:
    a[a<-180] = 360 + a[a<-180]
    a[a>180] = a[a>180] - 360
for i, (a, ax) in enumerate(zip(angles, angles_xformed)):
    *_, l = plt.hist(a, histtype='step', density=True, label=labels[i], lw=1)
    plt.hist(ax, histtype='step', density=True, lw=2.5, ls=':', color=l[0].get_edgecolor())
# plt.hist(angles, histtype='step', density=True, label=labels, lw=2)
# plt.hist(angles_xformed, histtype='step', density=True, label=labels[:2], lw=2, ls=':')
plt.xlabel('Angle /$\degree$ of supervisory action on x/y plane')
plt.ylabel('Probability density')
plt.legend(loc='lower center', ncol=3)

In [ ]:
# ... continued: Cosine similarity between shifted histograms
plt.figure(figsize=(4,2))
dist = []
step, bins = 2, 20
hist, *_ = np.histogram(angles[0], bins=bins, density=True)
for i in range(-180, 180, step):
    ar = rotate_deg(angles[1], i)
    arh, *_ = np.histogram(ar, bins=bins, density=True)
    dist.append(np.dot(arh, hist))
plt.plot(range(-180, 180, step), dist, label=labels[0])

dist = []
hist, *_ = np.histogram(angles[1], bins=bins, density=True)
for i in range(-180, 180, step):
    ar = rotate_deg(angles[0], i)
    arh, *_ = np.histogram(ar, bins=bins, density=True)
    dist.append(np.dot(arh, hist))
plt.plot(range(-180, 180, step), dist, label=labels[1])

plt.ylabel('Cosine similarity')
plt.yticks([])
plt.xlabel('Angle shift in supervisory actions / $\degree$')
plt.legend(loc='lower center', ncol=2)

In [ ]:
def rot_matrix(ang: float):
    ang = ang * np.pi / 180.
    mat = np.asarray([
        [np.cos(ang), -np.sin(ang), 0],
        [np.sin(ang), np.cos(ang), 0],
        [0, 0, 1]
    ])
    return mat.astype(np.float32)

In [ ]:
source_study_name = get_study(study_names[0])
_, source_agent, source_params = get_study_agent_params(source_study_name)

target_study_name = get_study(study_names[1])
_, target_agent, target_params = get_study_agent_params(target_study_name)

In [ ]:
xform_agent = transform_rl_policy(source_agent, np.zeros((3,12)), rot_matrix(-90), copy=True)

def _env_fn(own_params, target_params):
    params = deepcopy(source_params)
    env = get_env(scurve=False, disturbance_fn=make_disturbance_fn(target_heading))
    apply_params(env, **own_params)
    return env
s, t, x = run_comparison(
    env_fns = [lambda: _env_fn(sp, target_params) for sp in (source_params, target_params, source_params)],
    agents = [source_agent, target_agent, xform_agent],
    n=10
)
df = pd.DataFrame(dict(Source=s, Target=t, Transformed=x))
df.plot.density(figsize=(6,3))
plt.xlabel('Cumulative reward')

In [ ]:
plt.quiver(np.zeros(100), np.zeros(100), actions[1][:,0], actions[1][:,1], angles='xy', scale_units='xy', scale=1)
plt.gca().set_xlim(-env_rl.action_range[0], env_rl.action_range[0])
plt.gca().set_ylim(-env_rl.action_range[1], env_rl.action_range[1])

## Comparison

#### Parameters

In [ ]:
# Comparison of RL/PID parameters w/o wind
pidstudy = get_study_pid('MultirotorTrajEnvPID')
df_nominal = pidstudy.trials_dataframe().sort_values('value', ascending=False)
df_nominal = df_nominal.reset_index(drop=True).filter(like='params_').iloc[:10]

pidstudy = get_study_pid('MultirotorTrajEnvPIDWind5@90')
df_dist = pidstudy.trials_dataframe().sort_values('value', ascending=False)
df_dist = df_dist.reset_index(drop=True).filter(like='params_').iloc[:10]

In [ ]:
df_agg = pd.concat({'Nominal': df_nominal.mean(), 'Wind':df_dist.mean()}, axis=1)
df_agg.rename(index={k:k.replace('params_','') for k in df_agg.index.values}, inplace=True)
df_agg['Nominal $\sigma$ /%'] = df_nominal.std().values * 100 / df_agg['Nominal'].values
df_agg['Wind $\sigma$ /%'] = df_dist.std().values * 100 / df_agg['Wind'].values
df_agg = df_agg.reindex(sorted(df_agg.columns), axis=1)
df_agg['Change /%'] = (df_agg['Wind'] - df_agg['Nominal']) * 100. / df_agg['Nominal']
df_agg['Notable'] = (df_agg['Change /%'].abs() > df_agg['Nominal $\sigma$ /%'].abs()) & \
                    (df_agg['Change /%'].abs() > df_agg['Wind $\sigma$ /%'].abs())
df_agg.index = pd.MultiIndex.from_tuples([i.split('.') for i in df_agg.index], names=('Controller', 'Params'))
df_agg

In [ ]:
from IPython.display import display, Latex
Latex(df_agg.to_latex(float_format='%.3f', longtable=False))

#### Importances

In [ ]:
df = None
for study_name, study_name_pid in tqdm(zip(study_names, study_names_pid),
        leave=False, total=len(study_names)):
    study = get_study(study_name)
    imp = optuna.importance.get_param_importances(study)
    if df is None:
        df = pd.DataFrame(
            index=pd.Index(imp.keys(), name='Parameters'),
            columns=pd.MultiIndex.from_product([('RL', 'PID'), headings])
        )
    df[('RL', study.user_attrs.get('wind', '0@0'))] = pd.Series(imp)
    
    study_pid = get_study_pid(study_name_pid)
    imp_pid = optuna.importance.get_param_importances(study_pid)
    imp_pid = {('pid-'+k):v for k,v in imp_pid.items()}
    df[('PID', study_pid.user_attrs.get('wind', '0@0'))] = pd.Series(imp_pid)
df

In [ ]:
%matplotlib inline
df_ = df.sort_values(by=[('PID', '0@0')], ascending=False)
plt.figure(figsize=(15,3))
l, = plt.plot(df_[('RL','5@90')], label=labels[0])
plt.plot(df_[('PID','5@90')], label='PID' + labels[0][2:], c=l.get_c(), ls=':')
l, = plt.plot(df_[('RL','5@0')], label=labels[1])
plt.plot(df_[('PID','5@0')], label='PID' + labels[1][2:], c=l.get_c(), ls=':')
l, = plt.plot(df_[('RL','0@0')], label=labels[2])
plt.plot(df_[('PID','0@0')], label='PID' + labels[2][2:], c=l.get_c(), ls=':')
plt.legend(ncol=3)
plt.xticks(ticks=range(len(df_)),
           labels=params_to_labels(df_.index), rotation=25, ha='right');

In [ ]:
df_[[('PID','0@0'), ('PID','5@90'), ('RL','0@0'), ('RL','5@90')]]

#### Rewards

In [ ]:
# controllers on a single disturbance - density plot
def _env_rl():
    env = get_env(scurve=False,
              disturbance_fn=make_disturbance_fn('5@90'))
    apply_params(env, **best_params)
    return env
def _env_pid():
    env = get_env(scurve=False,
              disturbance_fn=make_disturbance_fn('5@90'))
    apply_params_pid(env.ctrl, **best_pid_params)
    return env
r_agent, r_pid = run_comparison(
    [_env_rl, _env_pid], (best_agent, None), n=30)

df = pd.DataFrame(dict(RL=r_agent, PID=r_pid))
df.plot.density(figsize=(6,3))
plt.xlabel('Cumulative reward')

#### Robustness

In [ ]:
# controllers vs disturbances - polar plot
angles_ = np.asarray(range(0, 360, 45))
df = pd.DataFrame(index=pd.Index(angles_, name='Heading'),
                  columns=pd.MultiIndex.from_product((('RL', 'PID'), labels[3:], ('Mean', 'Std'))))
for label, heading, study_name, study_name_pid in \
        tqdm(zip(labels, headings, study_names, study_names_pid),
        total=len(headings), leave=False):
    
    _, best_agent, best_params = get_study_agent_params(study_name)
    _, best_pid_params = get_study_params_pid(study_name_pid)

    for angle in tqdm(angles_, leave=False):
        env = get_env(scurve=False, disturbance_fn=make_disturbance_fn('5@%.2f' % angle))
        apply_params(env, **best_params)
        r = []
        for _ in range(10):
            env.reset()
            log = run_trajectory(env, wp, best_agent, reset_zero=False)
            r.append(log.reward.sum())
        df.loc[angle, ('RL', label[3:], 'Mean')] = np.mean(r)
        df.loc[angle, ('RL', label[3:], 'Std')] = np.std(r)
        
        env = get_env(scurve=False, disturbance_fn=make_disturbance_fn('5@%.2f' % angle))
        apply_params_pid(env.ctrl, **best_pid_params)
        r = []
        for _ in range(10):
            env.reset()
            log = run_trajectory(env, wp, None, reset_zero=False)
            r.append(log.reward.sum())
        df.loc[angle, ('PID', label[3:], 'Mean')] = np.mean(r)
        df.loc[angle, ('PID', label[3:], 'Std')] = np.std(r)
    

from itertools import product
df_ = df.copy()
df_.loc[360] = df.loc[0]

In [ ]:
lines = []
polys = []
for idx in product(*df_.columns.levels[:2]):
    l, = plt.polar(df_.index * np.pi / 180, df_[idx].Mean,
                   ls=':' if idx[0]=='PID' else '-', label=' '.join(idx))
    p = plt.fill_between(df_.index * np.pi / 180, df_[idx].Mean-df_[idx].Std,
                         df_[idx].Mean+df_[idx].Std, alpha=0.15, color=l.get_color(),
                         hatch='.' if idx[0]=='PID' else '-')
    polys.append(p)
    lines.append(l)
for l1, l2, p1, p2 in zip(lines[:3], lines[3:], polys[:3], polys[3:]):
    l2.set_color(l1.get_color())
    p2.set_color(p1.get_facecolor())
plt.legend(loc='lower center', ncol=3)

In [ ]:
lines = []
polys = []
kind = 'PID'
for idx in product(*df_.columns.levels[1:2]):
    l, = plt.polar(df_.index * np.pi / 180, df_[(kind, idx[0])].Mean,
                   ls=':' if kind=='PID' else '-', label=' '.join(idx))
    p = plt.fill_between(df_.index * np.pi / 180, df_[(kind, idx[0])].Mean-df_[(kind, idx[0])].Std,
                         df_[(kind, idx[0])].Mean+df_[(kind, idx[0])].Std, alpha=0.15, color=l.get_color(),
                         hatch='.' if kind=='PID' else '-')
    polys.append(p)
    lines.append(l)
for l1, l2, p1, p2 in zip(lines[:3], lines[3:], polys[:3], polys[3:]):
    l2.set_color(l1.get_color())
    p2.set_color(p1.get_facecolor())
plt.legend(loc='lower center', ncol=3)
plt.title(kind)

#### Long Trajectory

In [ ]:
study_name_pid, study_name = study_names_pid[1], study_names[1]

_, best_agent, best_params = get_study_agent_params(study_name)
_, best_pid_params = get_study_params_pid(study_name_pid)
heading = '0@0'

env = get_env(scurve=False, disturbance_fn=make_disturbance_fn('0@0'))
apply_params(env, **best_params)
log_rl = run_trajectory(env, nasa_wp[:4], best_agent)
print(len(log_rl), log_rl.position[-1])
env = get_env(scurve=False, disturbance_fn=make_disturbance_fn('5@0'))
apply_params(env, **best_params)
log_rl = run_trajectory(env, nasa_wp[4:], best_agent, log=log_rl, reset_zero=False)
print(len(log_rl), log_rl.position[-1])

env = get_env(scurve=False, disturbance_fn=make_disturbance_fn('0@0'))
apply_params_pid(env.ctrl, **best_pid_params)
log_pid = run_trajectory(env, nasa_wp[:4], None)
env = get_env(scurve=False, disturbance_fn=make_disturbance_fn('5@0'))
apply_params_pid(env.ctrl, **best_pid_params)
log_pid = run_trajectory(env, nasa_wp[4:], None, log=log_pid, reset_zero=False)

In [ ]:
%matplotlib notebook
fig = plt.figure()
ax = fig.add_subplot(projection='3d')
plt.plot(log_pid.x, log_pid.y, log_pid.z, label='PID', lw=0.75, ls='-')
plt.plot(log_rl.x, log_rl.y, log_rl.z, label='RL', lw=0.75, ls='-.')
plt.plot(nasa_wp[4,0],nasa_wp[4,1],nasa_wp[4,2], marker='x', c='b', lw=3, markersize=15)
plt.plot(log_pid.position[-1,0],log_pid.position[-1,1],log_pid.position[-1,2], marker='x', c='r', lw=3, markersize=15)
plt.gca().set_aspect('auto', 'box')
if not heading.startswith('0'):
    wx, wy, wz, dx, dy, dz = get_wind_quiver(heading, plt.gca(), n=5, dim=3)
    ax = plt.gca()
    ax.quiver(wx, wy, wz, 10*dx, 10*dy, dz, color='k', linewidth=0.75)
plt.legend()
plt.xlabel('x /m')
plt.ylabel('y /m')
ax.set_zlabel('z /m')
ax.view_init(elev=30, azim=45)

#### Wind magnitude vs failure rates

In [ ]:
# RL
study_name_pid, study_name = study_names_pid[2], study_names[2]

_, best_agent, best_params = get_study_agent_params(study_name)
_, best_pid_params = get_study_params_pid(study_name_pid)

winds = np.arange(0, 12, 2)
headings = np.arange(0, 360, 90)
H = {}
for h in tqdm(headings, leave=False):
    R, S = [], []
    for w in tqdm(winds, leave=False):
        heading = '%d@%d' % (w,h)
        env = get_env(scurve=False, disturbance_fn=make_disturbance_fn(heading))
        apply_params(env, **best_params)
        r = []
        for _ in trange(5, leave=False):
            env.reset()
            log = run_trajectory(env, wp, best_agent, reset_zero=False)
            r.append(log.reward.sum())
        R.append(np.mean(r))
        S.append(np.std(r))
    H[h] = (R, S)

In [ ]:
# PID
study_name_pid, study_name = study_names_pid[2], study_names[2]

_, best_agent, best_params = get_study_agent_params(study_name)
_, best_pid_params = get_study_params_pid(study_name_pid)

winds = np.arange(0, 12, 2)
headings = np.arange(0, 360, 90)
Hpid = {}
for h in tqdm(headings, leave=False):
    R, S = [], []
    for w in tqdm(winds, leave=False):
        heading = '%d@%d' % (w,h)
        env = get_env(scurve=False, disturbance_fn=make_disturbance_fn(heading))
        apply_params_pid(env.ctrl, **best_pid_params)
        r = []
        for _ in trange(5, leave=False):
            env.reset()
            log = run_trajectory(env, wp, None, reset_zero=False)
            r.append(log.reward.sum())
        R.append(np.mean(r))
        S.append(np.std(r))
    Hpid[h] = (R, S)

In [ ]:
%matplotlib inline
from matplotlib.lines import Line2D
plt.figure(figsize=(6,3))
for k in H.keys():
    m, s = np.asarray(H[k][0]), np.asarray(H[k][1])
    mpid, spid = np.asarray(Hpid[k][0]), np.asarray(Hpid[k][1])
    l, = plt.plot(winds, m, label=('$%d\degree$' % k))
    plt.fill_between(winds, m+s, m-s, color=l.get_color(), alpha=0.30)
    plt.plot(winds, mpid, ls=':', c=l.get_color())
    plt.fill_between(winds, mpid+spid, mpid-spid, color=l.get_color(), alpha=0.15, hatch='.')
plt.xlabel('Wind /N')
plt.ylabel('Reward over trajectory')
linep, liner = Line2D([0], [0], c='k', ls=':', label='PID'), Line2D([0], [0], c='k', ls='-', label='RL')
legend = plt.legend(handles=[linep, liner], ncol=2, loc='upper right');
plt.legend(ncol=4)
plt.gca().add_artist(legend)

In [ ]:
headings = np.arange(0, 360, 90)
top = pd.MultiIndex.from_product((('RL','PID'), [r'$%d\degree$'%h for h in headings]))
side = pd.Index(winds, name='Wind /N')
df = pd.DataFrame(index=side, columns=top)
df

for i, w in enumerate(winds):
    for k, h in enumerate(headings):
        df.loc[w, ('RL', r'$%d\degree$'%h)] = '$%.2f\pm%.2f$' % (H[h][0][i]/1000, H[h][1][i]/1000)
        df.loc[w, ('PID', r'$%d\degree$'%h)] = '$%.2f\pm%.2f$' % (Hpid[h][0][i]/1000, Hpid[h][1][i]/1000)
df

In [ ]:
print(df.style.to_latex(
    position='ht',
    hrules=True,
    caption='Tabulated results of performance degradation with increasing wind disturbance from different headings.'
))

In [ ]:
%matplotlib notebook
hh, ww = np.meshgrid([*headings, 0], winds, indexing='ij')
x = ww * np.cos(hh * np.pi / 180)
y = ww * np.sin(hh * np.pi / 180)
z_ = np.asarray([H[k][0] for k in H.keys()])
z = np.vstack((z_, z_[0]))

fig = plt.figure()
ax = fig.add_subplot(projection='3d')
ax.plot_surface(x, y, z, alpha=0.3)